# Smoothness as a function of # training samples

In [ ]:
import os
import sys
# If we don't need CUDA, do this before importing TF
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
import numpy as np
import pandas as pd
import tqdm
import tqdm.notebook
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
sns.set()

os.chdir("/nfs/scistore12/chlgrp/vvolhejn/smooth/logs/")

In [ ]:
%load_ext autoreload
%aimport smooth.config
%aimport smooth.datasets
%aimport smooth.model
%aimport smooth.analysis
%aimport smooth.callbacks
%aimport smooth.measures
%aimport smooth.util
%autoreload 1

In [ ]:
def plot_measure(ms, measure_name):
    grid = sns.relplot(
        data=ms,
        x="dataset.samples_train",
        y=measure_name,
        hue="model.hidden_size",
        palette=smooth.analysis.make_palette(ms["model.hidden_size"]),
#         kind="line",
    )

    ax = grid.axes[0][0]
    if "loss_train" in measure_name:
        ax.set_yscale("log")
    ax.set_xscale("log")
#     ax.set_xlim((3, 1200))
    plt.show()

def plot_measures(ms):    
    for measure in [
        "loss_train", "loss_test",
    #     "gradient_norm_train",
        "gradient_norm_test",
        "weights_product",
        "path_length_f_test",
        "path_length_d_test",
    ]:
        plot_measure(ms, measure)

In [ ]:
ms = pd.read_feather("./0406_mnist_binary_increasing/measures.feather")
smooth.analysis.remove_constant_columns(ms, verbose=True)

ms.head()
plot_measures(ms)

## Re-do

In [ ]:
ms = smooth.analysis.load_measures(
      "./0519_binary_increasing/measures.feather",
#     "./0518_binary_increasing/measures.feather",
    kind_cols=[("dataset.samples_train", "samples")],
    remove_unconverged=True,
)
ms["model.hidden_size"] = 256

taus, ps = [
    smooth.analysis.summarize_kendalls(
        ms,
        groupby="dataset.name",
        x_col="dataset.samples_train",
        y_cols=smooth.analysis.get_measure_names(),
        get_pvalues=get_pvalues,
    )
    for get_pvalues in [False, True]
]
display(taus.describe().round(2))
display(ps.describe())

In [ ]:
ms1 = ms[ms["dataset.name"] == "mnist28"]
import scipy.stats
scipy.stats.kendalltau(ms1["dataset.samples_train"], ms1["weights_product"])

In [ ]:
scipy.stats.kendalltau([0, 0, 1, 1], [0, 0, 1, 1])

In [ ]:
def format_for_table(col_dfs):
    measures = [
        ("gradient_norm_test", "GN"),
        ("weights_product", "WP"),
        ("path_length_f_test", "PL_0"),
        ("path_length_d_test", "PL_1"),
    ]
    
    rows = []
    
    for measure_col, measure_tex in measures:
        row = "${}$".format(measure_tex).ljust(10)
        
        for col_df in col_dfs:
            row += " & ${:.2f} \pm {:.2f}$".format(
                col_df.loc["mean", measure_col],
                col_df.loc["std", measure_col],
            )
        rows.append(row)
    
    for row in rows:
        print(row)

format_for_table([taus.describe(), ps.describe()])

In [ ]:
taus

In [ ]:
for measure in [
    "loss_train", "loss_test",
#     "gradient_norm_train",
    "gradient_norm_test",
    "weights_product",
    "path_length_f_test",
    "path_length_d_test",
    "actual_epochs",
]:
    plot_measure(ms[ms["dataset.name"] == "mnist59"], measure)

## Re-do 2 (init scale 1.0)

In [ ]:
ms = smooth.analysis.load_measures(
#       "./0523_binary_increasing_is_1/measures.feather",
#       "./0525_binary_increasing_is_1/measures.feather",
      "./0528_binary_increasing_is_1/measures.feather",
    kind_cols=[("dataset.samples_train", "samples")],
    remove_unconverged=True,
)
ms["model.hidden_size"] = 256

taus, ps = [
    smooth.analysis.summarize_kendalls(
        ms,
        groupby="dataset.name",
        x_col="dataset.samples_train",
        y_cols=smooth.analysis.get_measure_names(),
        get_pvalues=get_pvalues,
    )
    for get_pvalues in [False, True]
]
display(taus.describe().round(2))
display(ps.describe())

In [ ]:
format_for_table([taus.describe()])

In [ ]:
for measure in [
    "loss_train", "loss_test",
#     "gradient_norm_train",
    "gradient_norm_test",
    "weights_product",
    "path_length_f_test",
    "path_length_d_test",
    "actual_epochs",
]:
    plot_measure(ms, measure)

In [ ]:
for measure in [
    "loss_train", "loss_test",
#     "gradient_norm_train",
    "gradient_norm_test",
    "weights_product",
    "path_length_f_test",
    "path_length_d_test",
    "actual_epochs",
]:
    plot_measure(ms, measure)